In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Normalize to [0, 1]
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(pareto_front)

# Split data (80% train, 20% validation)
X_train, X_val = train_test_split(X_normalized, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Define autoencoder architecture
input_dim = 2
latent_dim = 1  # 1D latent space for simplicity

# Encoder
inputs = Input(shape=(input_dim,))
encoded = Dense(32, activation='relu')(inputs)
encoded = Dense(latent_dim, activation='linear')(encoded)

# Decoder
decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Compile
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Train
history = autoencoder.fit(
    X_train, X_train,
    epochs=500,
    batch_size=16,
    validation_data=(X_val, X_val),
    verbose=1
)

In [ ]:
# Encode two Pareto solutions
z_A = encoder.predict(X_train[0:1])  # Solution A
z_B = encoder.predict(X_train[1:2])  # Solution B

# Linear interpolation
alpha = 0.5
z_new = alpha * z_A + (1 - alpha) * z_B

# Decode to generate new solution
J_new = decoder.predict(z_new)

# Denormalize
J_new_original = scaler.inverse_transform(J_new)
print(f"Interpolated Solution: Time = {J_new_original[0, 0]:.2f} s, Energy = {J_new_original[0, 1]:.2f} kWh")